### Imports

In [ ]:
import sqlalchemy as db
from sqlalchemy import Column, Integer, String, ForeignKey, Float, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.sql.schema import UniqueConstraint
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlite3 import IntegrityError

import bs4
from bs4 import BeautifulSoup
import requests
import json
from json import JSONDecodeError
import os
import subprocess
import pandas as pd
import pprint
import time
import sys
import pickle

print('\033[1mVersions:\033[0m')
print('bs4:\t\t', bs4.__version__)
print('requests:\t', requests.__version__)
print('pandas:\t\t', pd.__version__)
print('json:\t\t', json.__version__)
print('sqlalchemy\t', db.__version__)

## Getting product id

In [ ]:
def get_id(url):
    url_splitted = url.split('/')
    return url_splitted[-1]

## Getting product experience name

In [ ]:
def get_experience(soup):
    scripts = soup.findAll('script')

    for script in scripts:
        try:
            text = script.get_text()
            index = text.find('{"experience":"')
            if index != -1:
                text = text[index:index+30]
                text = text.split('"')
                text = text[3:]
                text = text[0]
                return text
            else:
                pass
        except JSONDecodeError:
            print('--> JSONDecodeError in:', text)

## Getting product price(s)

In [ ]:
def get_prices(soup, product_id, experience_name, branch, sku):
    
    returning_list = []
    
    header = {"accept":"*/*","accept-language":"pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7","content-type":"application/json","sec-fetch-dest":"empty","sec-fetch-mode":"cors","sec-fetch-site":"same-origin","wm_qos.correlation_id":"96af80b1-023-170cc42c494ea9,96af80b1-023-170cc42c494e4f,96af80b1-023-170cc42c494e4f"}

    dict_data = {                                        
          "fsa": "P7B",                                  
          "products": [{                                            
              "productId": product_id,         
              "skuIds": [sku]                                         
            }],                                             
          "lang": "en",                                  
          "pricingStoreId": branch,          
          "fulfillmentStoreId": branch,      
          "experience": experience_name
    }
    
    dict_data = json.dumps(dict_data)

    session = requests.Session()
    session.headers.update(header)
    response = session.post('https://www.walmart.ca/api/product-page/v2/price-offer', headers=header, data=dict_data)
    response_dict = json.loads(response.text)
    
    skus_name = response_dict['skus'].values()
    skus_name = list(skus_name)[0]

    for sku in skus_name:
        temp_dict = {}
        s = response_dict['offers'][sku]
        temp_dict['price'] = s['currentPrice']
        temp_dict['availability'] = s['gmAvailability']
        temp_dict['stock'] = s['availableQuantity']
        temp_dict['sku'] = sku
        
        returning_list.append(temp_dict)
        
    return returning_list

## Getting product barcode

In [ ]:
def get_barcode(soup):
    scripts = soup.findAll('script')

    for script in scripts:
        try:
            text = script.get_text()
            index = text.find('"upc":["')
            if index != -1:
                text = text[index:index+21]
                text = text.split('"')
                return text[3]
        except JSONDecodeError:
            print('JSONDecodeError in:', text)

## Getting package

In [6]:
def get_package(soup):
    return soup.p.get_text()

## Product category

In [7]:
def get_category(soup):
    category = soup.ol.get_text()
    category = category.split('›')
    category = category[1:-1]
    category = '|'.join(category)
    return category

### Function to get product info

In [8]:
def get_data_by_url(url, branch):
    return_dict = {}        
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    scripts = soup.findAll('script')
        
    for script in scripts:
        try:
            text = script.get_text()
            content = json.loads(text)

            product_id = get_id(url)
            return_dict['id'] = product_id
            return_dict['name'] = content['name']
            return_dict['brand'] = content['brand']['name']
            return_dict['description'] = content['description']
            return_dict['package'] = get_package(soup)
            return_dict['images'] = content['image']
            return_dict['category'] = get_category(soup)
            return_dict['sku'] = content['sku']
            return_dict['barcode'] = get_barcode(soup)

            return_dict['prices'] = get_prices(soup,product_id,get_experience(soup),branch,content['sku'])
            
            return_dict['branch'] = branch

            return return_dict
        
        except KeyError:
            pass
        except JSONDecodeError:
            pass
        except Exception as e:
            print(format(e), type(e))
            return None

### Testing the first page and collecting the redirecting links

In [9]:
def bold(string):
    return f'\033[1m{string}\033[0m'

In [10]:
cookies = {
    'enableHTTPS': '1',
    'userSegment': '40-percent',
    'deliveryCatchment': '3124',
    'vtc': 'cgt77dih2FdxATfS4uUDUo',
    'walmart.nearestPostalCode': 'P7B3Z7',
    'walmart.nearestLatLng': '48.4120872,-89.2413988',
    'rxVisitor': '15837802237056RTP834MTTFN5C9RM8UJMDF0G5TNOV4I',
    'walmart.id': '98aa61a9-0ef2-440c-907f-8e35a03d4040',
    'walmart.shippingPostalCode': 'P7B3Z7',
    'defaultNearestStoreId': '3124',
    'zone': '9',
    'walmart.csrf': '845346c6b117421dfadb8e3f',
    'wmt.c': '0',
    'DYN_USER_ID': '1cd8c327-e36c-4c30-ae31-450ffa145e6c',
    'WM_SEC.AUTH_TOKEN': 'MTAyOTYyMDE4JGp4bHoPPqM4IHwXXolda5CQxQid4PyYiM+D/nJqGkYOKrkubp5nLx5jSnhK/qu9oeqlDi8Y23SNoqbm43s7PvfOaZ/7RXaPhEvmbftJidzPNZe60HOO8d5KX9YgFsTqj8OFN4dileb20bpDLeCIlSFd/Hsc7bnSe4+TLU2zbj15L8sPl1ruR+2Co0+FoYvKAzEtT/9xYxFbQNViJ74oNszb+4ZNSjjfJDllwsNoIxbb/SoGFgAYL9DGZ8K45WCXb/Ew67/GsLtdlJHpe1JgEK6sU4RxgKKclgqkZSx5MDDt6MEVBaCttxNgmVmkO5DNgU2ycpstumrMJ8j6R+zbrTgp1zl+wwoVzsgI+vuWXwmdTvDsR7ns23LRp9X21gEUypNl3y1IM7t+ItR8jbJL5w==',
    'LT': '1583780225166',
    's_ecid': 'MCMID%7C52763711784505000869214469084195096524',
    'AMCVS_C4C6370453309C960A490D44%40AdobeOrg': '1',
    'wmt.breakpoint': 'd',
    'DYN_USER_ID.ro': '1cd8c327-e36c-4c30-ae31-450ffa145e6c',
    'TBV': '7',
    'dtSa': '-',
    'dtLatC': '50',
    'usrState': '1',
    'previousBreakpoint': 'desktop',
    'walmart.locale': 'en',
    'og_session_id': 'af0a84f8847311e3b233bc764e1107f2.711939.1583780282',
    'og_session_id_conf': 'af0a84f8847311e3b233bc764e1107f2.711939.1583780282',
    'cartId': '5377abc4-dbb7-4cbb-a45f-52664abbf17c',
    'NEXT_GEN.ENABLED': '1',
    'og_autoship': '0',
    's_cc': 'true',
    'BVBRANDID': '819bff00-6fc8-4f17-9063-6e35e8edf3d8',
    'BVImplmain_site': '2036',
    'cookiePolicy': 'true',
    's_visit': '1',
    'headerType': 'whiteGM',
    'localStoreInfo': 'eyJwb3N0YWxDb2RlIjoiUDdCM1o3IiwibG9jYWxTdG9yZUlkIjoiMzEyNCIsInNlbGVjdGVkU3RvcmVJZCI6IjMxMjQiLCJzZWxlY3RlZFN0b3JlTmFtZSI6IlRodW5kZXIgQmF5IFN1cGVyY2VudHJlIiwiZnVsZmlsbG1lbnRTdG9yZUlkIjoiMzEyNCIsImZ1bGZpbGxtZW50VHlwZSI6IklOU1RPUkVfUElDS1VQIn0K',
    's_sq': '%5B%5BB%5D%5D',
    'bstc': 'Z6j5qTjOWS1tUBGFnJyyfI',
    'xpa': '2lwWQ|6FUTl|JFA3E|LVSOt|MZ9tt|NOECn|NOaJP|O7oLY|OnAAN|SXgTw|WsUbN|ZINk3|_AAED|jeBOs|mOlOu|qgEW6|sGGbM|xzlm3|yI7_k',
    'exp-ck': '2lwWQi6FUTl1JFA3E3LVSOt1MZ9tt1NOECn2NOaJPxO7oLY1OnAAN1SXgTw6WsUbN1ZINk34_AAED6jeBOs5mOlOuCqgEW61sGGbM4xzlm31yI7_k1',
    'TS011fb5f6': '0130aff232a74f3092f0512a029d6a5a91c9a8220a9667fd296ef66ea1ac8020bee7d0b55f71f9ec213326c98e8685ecaf2daddc97',
    'TS0175e29f': '0130aff232a74f3092f0512a029d6a5a91c9a8220a9667fd296ef66ea1ac8020bee7d0b55f71f9ec213326c98e8685ecaf2daddc97',
    '_ga': 'GA1.2.2103727771.1583927388',
    '_gid': 'GA1.2.616863720.1583927389',
    'xpm': '1%2B1583926997%2Bcgt77dih2FdxATfS4uUDUo~%2B0',
    'akaau_P1': '1583929189~id=3743fe117eba5e189e2f81ea8d765cf4',
    'AMCV_C4C6370453309C960A490D44%40AdobeOrg': '-432600572%7CMCIDTS%7C18333%7CMCMID%7C52763711784505000869214469084195096524%7CMCAID%7CNONE%7CMCOPTOUT-1583934588s%7CNONE%7CvVersion%7C4.5.2%7CMCAAMLH-1584532188%7C4%7CMCAAMB-1584532188%7Cj8Odv6LonN4r3an7LhD3WZrU1bUpAkFkkiY1ncBR96t2PTI%7CMCSYNCSOP%7C411-18340',
    '_gcl_au': '1.1.1253472050.1583927392',
    '_fbp': 'fb.1.1583927392090.1342832719',
    'BVBRANDSID': '38af1296-f393-4fb1-830a-b708f1d7f10d',
    'rxvt': '1583929201781|1583927386468',
    'dtPC': '39$527385326_169h1p39$527386849_816h1p39$527390902_418h1p39$527401744_838h1vGPAKMHBORGNEPCNWOUDOFONEOPAPVETL',
    'authDuration': '{"lat":"1583927405573000","lt":"1583927405573000"}',
    'dtCookie': '9$7B89556EA0A2938FEE1966C08A1BE7B9',
    'ENV': 'ak-dfw-prod',
    'seqnum': '19',
    'TS01f4281b': '0130aff2324b75e0a5578489038b0889933ed5ae482e7353d43029fd76243aaacaeaedd669342325a1e5aee489db1ec44543948043',
    's_gnr': '1583929038321-Repeat',
    '_4c_': 'jVRdT%2BQgFP0rmz74JFNogYKJMeq62d34rck%2BNkzBKbFTGqBTXTP%2FfS%2FjOOommywPlHs494MDty%2FZ1Jo%2BOyBMlLKQuJTw2c8ezXPIDl6yZkjzKk2j77KDrI1xCAd5Pk3TbFLdUvk4a1Ru%2BnxQffTPqHVjMK3rdECDiSHf2SiMw9BZE%2FJHGxtIiprOqN72i8%2BA8SG%2FRCXBPNvPGqcNZCVyJmYM7PgbLEQEhvXgnR6bWMfnIXEmM%2F8S9CNsaLOyjaknq2ObnEuO39HW2EUbAa64SOjgYY1n%2FHUIzkoqBEm5J9trN%2B3CFJjid3QXhmBKAZ57NwWTYp223i3Nl02JDkTMzm0%2FPoHhzYPxfsP5t4pujNo5jzq7SsoMKlqHHkbf2zh6A8IUTJIj21h92EKaQS1M%2Ff26vjMquF519amKZuH8c31vO1NfJ%2Ff625t7%2FesC6gg2bvTa5d1icOcAJ2FXSR5YdK5RXeLCC9nPri7ub%2BuTs%2BPTq8sPR3DL19gohO7jWeZ5CPm0tI3qlVbprnKS%2F7xDxYyIGUbnx7dlHrgseYFxVVSYMnF0fHNySPaWcDZWVLysCKkEZZhhjAWXBaGUSywokQxLzgq6d3xzdkjeSrs4%2FfEVCvs%2FV3C693axMP7CxNZpcLz3SlvQC2RMDwOaAt7Mgxq7mMx0102nQrCNNuExuiFb72dPm86hmJSVZGUh4IVGaBPBKU4DGN7qbQtlc14INTcM8bKQiFaqQeCkUFVQMscPVEkls21MSRgDOpGSrl%2BLeY3xXtE7jdMSjgU0%2B8ZSf%2B0zwkWqZdoSdhsVlP6ZmhCgrnaxsJbVvCw54pppRE3DkRC8RETzOSWGsYbrt3SbfwhJStCtEtCqGyHW6z8%3D',
}

headers = {
    'authority': 'www.walmart.ca',
    'pragma': 'no-cache',
    'cache-control': 'no-cache',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36',
    'sec-fetch-dest': 'document',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
}

In [12]:
BRANCHES = ['https://www.walmart.ca/en/pantry-households-pets/drinks/energy-drinks/N-3124',
            'https://www.walmart.ca/en/pantry-households-pets/household-supplies/kitchen-cleaning/kitchen-cleaners/N-3106']

URL_MAIN =  'https://www.walmart.ca'

products_list = []
dataset = []

for branch in BRANCHES:
    
    branch_name = branch.split('/')[-1].split('-')[1]
    print(f'\nScraping branch: {branch_name}')

    response = requests.get(branch, headers=headers, cookies=cookies)

    soup = BeautifulSoup(response.content, 'html.parser')
    
    for i in soup.findAll('a'):
        href = i.get('href')
        if href is not None and '/en/ip/' in href:
            products_list.append(href)
    
    total = len(products_list)
    for i,href in enumerate(products_list):
        try:
            init = time.time()

            get_url = f'{URL_MAIN}{href}'
            new_product = get_data_by_url(get_url, branch_name)
            price = new_product['prices'][0]['price']
            dataset.append(new_product)

            end_time = round(time.time()-init, 2)
            print(f"[{i+1}/{total}] ({end_time}s) ${price} {bold(new_product['name'])}")
        except KeyboardInterrupt:
            break
        except Exception as e:
            print(f"[{i+1}/{total}] {type(e)}{format(e)}\n{bold('IN:')} URL: {get_url}")
    products_list = []
    


Scraping branch: 3124
[1/46] (2.6s) $10.47 MONSTER Green, 473mL, 4 pack
[2/46] (2.53s) $9.97 MONSTER ENERGY Zero Ultra, 473mL, 4 pack
[3/46] (2.71s) $16.97 Red Bull Energy Drink
[4/46] (2.94s) $16.97 Red Bull Energy Drink Sugarfree
[5/46] (2.82s) $8.97 Red Bull Energy Drink
[6/46] (2.4s) $2.77 Starbucks Frappuccino Vanilla
[7/46] (2.34s) $2.97 Starbucks Frappuccino Mocha
[8/46] (2.63s) $2.77 Starbucks Frappuccino Caramel
[9/46] (2.72s) $2.97 MONSTER ENERGY Green, 473mL, Can
[10/46] (2.84s) $3.17 MONSTER ENERGY Zero Ultra, 473mL, Can
[11/46] (2.57s) $3.47 Red Bull Energy Drink Original
[12/46] (2.39s) $8.97 Red Bull Energy Drink Sugarfree
[13/46] (2.55s) $16.97 5 Hour Energy Extra Strength Blue Raspberry
[14/46] (2.72s) $13.97 Red Bull Energy Drink
[15/46] (2.8s) $2.97 Starbucks Doubleshot White Chocolate
[16/46] (2.72s) $3.27 Red Bull Energy Drink Kiwi, Green Edition
[17/46] (2.6s) $2.67 Rockstar Sugar Free Energy Drink
[18/46] (2.41s) $3.47 Red Bull Energy Drink Sugarfree
[19/46] (2.

In [13]:
# Serializing dataset
pickle_out = open("dataset.pickle","wb")
pickle.dump(dataset, pickle_out)
pickle_out.close()
print(f'Number of products serialized: {len(dataset)}')

Number of products serialized: 31


## Visualization of dataset

In [18]:
df = pd.DataFrame(dataset)
del df['images']
del df['description']
del df['prices']
del df['category']
df.sample(10)

,id,name,brand,package,sku,barcode,branch
26,6000200672034,Rockstar XD Thermo Marshmallow,Rockstar,Rockstar XD Thermo Marshmallow 12x473mL,6000200672035,81809400788,3124
0,6000098847546,"MONSTER Green, 473mL, 4 pack",Monster Energy,4 x 473 mL,10217758,7084781232,3124
7,6000192741261,Starbucks Frappuccino Caramel,Starbucks,405mL,6000192741262,6900014896,3124
17,6000188742990,Red Bull Energy Drink Sugarfree,Red Bull,1 x 355 mL,6000188742991,18085400060,3124
1,6000188745003,"MONSTER ENERGY Zero Ultra, 473mL, 4 pack",Monster Energy,Monster Ultra 4pk,6000188745005,7084701882,3124
19,6000195949713,"Red Bull Energy Drink Tropical, Yellow Edition",Red Bull,4 x 250 mL,6000195949714,18085400012,3124
15,6000197987069,"Red Bull Energy Drink Kiwi, Green Edition",Red Bull,1 x 355 mL,6000197987070,18085400040,3124
13,6000188745789,Red Bull Energy Drink,Red Bull,4 x 355 mL,6000188745790,18085400041,3124
2,6000188742978,Red Bull Energy Drink,Red Bull,8 x 250 mL,6000188742979,18085400070,3124
23,6000133214838,"MONSTER ENERGY Lo-Carb, 473mL, Can",Monster Energy,473 mL,10279732,7084781125,3124
